In [10]:
import torch
from torchsummary import summary

from src.cvae import VariationalAutoEncoder
from src.ReciMeEncoder import ReciMeEncoder, RmeParameters

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
D_in = 10000
recimeEnc = ReciMeEncoder(RmeParameters(D_in)).to(device)
summary(recimeEnc, (D_in,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 4096]      40,964,096
       BatchNorm1d-2                 [-1, 4096]           8,192
              ReLU-3                 [-1, 4096]               0
ReLUBatchNormLinear-4                 [-1, 4096]               0
            Linear-5                 [-1, 2048]       8,390,656
       BatchNorm1d-6                 [-1, 2048]           4,096
              ReLU-7                 [-1, 2048]               0
ReLUBatchNormLinear-8                 [-1, 2048]               0
            Linear-9                 [-1, 1024]       2,098,176
      BatchNorm1d-10                 [-1, 1024]           2,048
             ReLU-11                 [-1, 1024]               0
ReLUBatchNormLinear-12                 [-1, 1024]               0
           Linear-13                  [-1, 512]         524,800
      BatchNorm1d-14               

In [3]:
D_in = 65608
H = 4096
H2 = 1024
latent_dim = 512
vae = VariationalAutoEncoder(D_in, H, H2, latent_dim).to(device)
summary(vae, (D_in,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 4096]     268,734,464
       BatchNorm1d-2                 [-1, 4096]           8,192
              ReLU-3                 [-1, 4096]               0
ReLUBatchNormLayer-4                 [-1, 4096]               0
            Linear-5                 [-1, 1024]       4,195,328
       BatchNorm1d-6                 [-1, 1024]           2,048
              ReLU-7                 [-1, 1024]               0
ReLUBatchNormLayer-8                 [-1, 1024]               0
            Linear-9                 [-1, 1024]       1,049,600
      BatchNorm1d-10                 [-1, 1024]           2,048
             ReLU-11                 [-1, 1024]               0
ReLUBatchNormLayer-12                 [-1, 1024]               0
           Linear-13                  [-1, 512]         524,800
      BatchNorm1d-14                  

In [3]:
import torch
import torchwordemb

_, out = torchwordemb.load_word2vec_bin('vocab.bin')
torch.save(out, 'embeddings.pt')

In [12]:
import word2vec
import sys
import os

'''
Usage: python get_vocab.py /path/to/vocab.bin
'''
w2v_file = 'data/vocab.bin'
model = word2vec.load(w2v_file)

vocab =  model.vocab

print("Writing to %s..." % os.path.join(os.path.dirname(w2v_file),'vocab.txt'))
f = open(os.path.join(os.path.dirname(w2v_file),'vocab.txt'),'w')
f.write("\n".join(vocab))
f.close()

Writing to data/vocab.txt...


In [22]:
print(type(model))
print(type(model.vectors))
print(model.vectors.shape)
print(model.vocab.shape)

<class 'word2vec.wordvectors.WordVectors'>
<class 'numpy.ndarray'>
(30167, 300)
(30167,)


In [5]:
D_in = 65608
latent_dim = 512
CnnVae = CnnVariationalAutoEncoder(D_in, latent_dim).to(device)
summary(CnnVae, (D_in,))

In [ ]:
# Ingr 1 Ingr2
["Apfel", "Brine"]
["Birne", "Traube"]

# Ingr1_Apfel Ingr1_Birne Ingr1_Traube Ingr2_Apfel Ingr2_Brine Ingr2_Traube
[1 0 0 0 1 0]
[0 1 0 0 0 1]

# Ingr 1 Ingr 2
# Apfel = 1 Brine = 2 Traube = 3
[1 2]
[2 3]

In [8]:
import pandas as pd
string = pd.DataFrame(["Test"], columns=["ingr"])
string.astype('int')

ValueError: invalid literal for int() with base 10: 'Test'

In [ ]:
import torch
from torchsummary import summary

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

D_in = 65608
latent_dim = 512

class CnnVariationalAutoEncoderNew(torch.nn.Module):
    def __init__(self, D_in, latentDim):
        super(CnnVariationalAutoEncoderNew, self).__init__()
        H1 = 32768
        self.C1 = (2048,32)
        C2 = (4096,16)
        C3 = (4096,8)
        C3 = (4096,2)
        H2 = 8192

        self.encoderStack_dense = torch.nn.Sequential(
            torch.nn.Linear(D_in, H1),
            torch.nn.BatchNorm1d(H1),
            torch.nn.ReLU(),
        )

        self.encoderStack_cnn = torch.nn.Sequential(
            torch.nn.Conv1d(self.C1[1], C2[1], kernel_size=16, padding=0, bias=False),
            torch.nn.ReLU(),
            torch.nn.AvgPool1d(2),
        )
    
    def encode(self, x):
        print(x.shape)
        dense = self.encoderStack_dense(x)
        print(dense.shape)
        reshaped = torch.reshape(dense, self.C1)
        return self.encoderStack_cnn(reshaped)

    def forward(self, x):
        return self.encode(x)

CnnVae = CnnVariationalAutoEncoderNew(D_in, latent_dim).to(device)

summary(CnnVae, tuple([D_in]))